<a href="https://colab.research.google.com/github/Z0rr09/OD-WeaponDetection/blob/master/Notebook_module_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
hallo


NameError: name 'hallo' is not defined

ca va ?
